# Setup Colab Environment

In [5]:
from google.colab import drive
drive.mount('/content/drive')
# %cd drive/MyDrive/CS523\ Work/
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 data/                           sketch_guided_t2i/
 neurips2022-nmmo-starter-kit/   stable_diffusion.ipynb
'Programming Exercise 1.ipynb'


In [101]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 5.8 MB 55.9 MB/s 
     |████████████████████████████████| 1.3 MB 67.1 MB/s 
     |████████████████████████████████| 182 kB 77.7 MB/s 
     |████████████████████████████████| 7.6 MB 60.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=22358cc8f42fae5a3b947c27ea2a06c1e7196d6afad314299c5a6a55517aa503
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


# Import

In [42]:
import torch
import tensorflow as tf
from PIL import Image
import numpy as np


ModuleNotFoundError: ignored

## Obtain Similarity Model

In [7]:
VGG16 = tf.keras.applications.VGG16(include_top=True, weights='imagenet')
x = VGG16.layers[-2].output
VGG16_MODEL = tf.keras.Model(inputs = VGG16.input, outputs = x)
VGG16_MODEL.trainable = False
VGG16_MODEL.summary()

553467096/553467096 [==============================] - 25s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

# Load the images

In [15]:
from PIL import Image
import glob
same_images_list = []
for dir in glob.glob("./sketch_guided_t2i/*"):
    lst = []
    for filename in glob.glob(dir + "/*.jpg"):
        im=Image.open(filename)
        lst.append(im)
    same_images_list.append(lst)

print(same_images_list[0])

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7F7CD06051C0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1328x1328 at 0x7F7CD0605070>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1026x1028 at 0x7F7CD0605130>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7F7C8E1A1D90>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7F7C8E0C5790>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7F7C8E0C5BB0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1026x1028 at 0x7F7C8E0C5EB0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7F7C8E0C9430>]


# Similarity metrics

## L2 Norm

In [61]:
# Helper functions
def image_to_feature_vector(img):
    return np.array(tf.squeeze(VGG16_MODEL(tf.expand_dims(img, 0))))

from numpy.linalg import norm
def similarity(v1, v2):
    v1 /= norm(v1)
    v2 /= norm(v2)
    return 1. - norm(v2 - v1)

In [17]:
def get_similarity(img1, img2):
  mean = np.array([0.485, 0.456, 0.406])
  std = np.array([0.229, 0.224, 0.225])

  image1 = tf.image.resize_with_pad(img1, 224, 224)
  # Normalize
  image1 = tf.math.divide(image1, 255.)
  image1 = tf.math.subtract(image1, mean)
  image1 = tf.math.divide(image1, std)
  image1 = image_to_feature_vector(image1)

  image2 = tf.image.resize_with_pad(img2, 224, 224)
  # Normalize
  image2 = tf.math.divide(image2, 255.)
  image2 = tf.math.subtract(image2, mean)
  image2 = tf.math.divide(image2, std)
  image2 = image_to_feature_vector(image2)

  sim = similarity(image1, image2)
  return sim

## Cosine similarity

In [18]:
from scipy.spatial.distance import cosine
def get_cosine_sim(img1, img2):
  return 1 - cosine(img1, img2)

## Inception score

In [19]:
# from tf.keras.applications.inception_v3 import InceptionV3
# from tf.keras.applications.inception_v3 import preprocess_input
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
# load inception v3 model
inception_model = tf.keras.applications.inception_v3.InceptionV3()
def calculate_inception_score(images, eps=1E-16):
    images = np.array(images)
    processed = images.astype('float32')
    # pre-process raw images for inception v3 model
    processed = tf.keras.applications.inception_v3.preprocess_input(processed)
    # predict class probabilities for images
    yhat = inception_model.predict(processed)
    # enumerate splits of images/predictions
    p_yx = yhat
    # calculate p(y)
    p_y = expand_dims(p_yx.mean(axis=0), 0)
    # calculate KL divergence using log probabilities
    kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
    # sum over classes
    sum_kl_d = kl_d.sum(axis=1)
    # average over images
    avg_kl_d = mean(sum_kl_d)
    # undo the log
    is_score = exp(avg_kl_d)
    return is_score

96112376/96112376 [==============================] - 5s 0us/step


In [114]:
def get_average_is(_images):
    inception_scores = []
    for _i in range(len(_images)):
        images = _images[_i]
        for i in range(len(images)):
            for j in range(i + 1, len(images)):
                img1 = images[i]
                img2 = images[j]
                size = 299, 299
                img1.thumbnail(size)
                img2.thumbnail(size)
                np_img1 = tf.keras.utils.img_to_array(img1)
                np_img2 = tf.keras.utils.img_to_array(img2)
                # print(np_img1.shape, np_img2.shape)
                np_img1 = tf.image.resize(np_img1, [224, 224])
                np_img2 = tf.image.resize(np_img2, [224, 224])
                is_score = calculate_inception_score([np_img1, np_img2])
                inception_scores.append(is_score)
    return np.mean(np.array(inception_scores)), np.std(np.array(inception_scores))

In [115]:
is_same_mean, is_same_std = get_average_is(same_images_list)

1/1 [==============================] - 0s 24ms/step


In [116]:
print("Mean inception score for same prompt pairs:", is_same_mean)
print("Standard deviation of the inception scores for same prompt pairs:", is_same_std)

Mean inception score for same prompt pairs: 1.7237407
Standard deviation of the inception scores for same prompt pairs: 0.32978928


# VGG16 Feature vector

In [102]:
import copy
def vgg16_similarity(_images):
    l2_sim_scores = []
    cosine_sim_scores = []
    size = (224, 224)
    for _i in range(len(_images)):
        images = _images[_i]
        for i in range(len(images)):
            for j in range(i + 1, len(images)):
                img1 = copy.deepcopy(images[i])
                img2 = copy.deepcopy(images[j])
                img1.thumbnail(size)
                img2.thumbnail(size)
                np_img1 = tf.keras.utils.img_to_array(img1)
                np_img2 = tf.keras.utils.img_to_array(img2)
                np_img1 = tf.image.resize(np_img1, [224, 224])
                np_img2 = tf.image.resize(np_img2, [224, 224])
                
                # L2 Norm
                l2_sim = get_similarity(np_img1, np_img2)
                # Cosine similarity
                cosine_sim = get_cosine_sim(image_to_feature_vector(np_img1), image_to_feature_vector(np_img2))
                l2_sim_scores.append(l2_sim)
                cosine_sim_scores.append(cosine_sim)

    return np.mean(np.array(l2_sim_scores)), np.std(np.array(l2_sim_scores)), np.mean(np.array(cosine_sim_scores)), np.std(np.array(cosine_sim_scores))

In [103]:
vgg_l2_same_mean, vgg_l2_same_std, vgg_cosine_same_mean, vgg_cosine_same_std = vgg16_similarity(same_images_list)

In [104]:
print("Mean L2 Norm similarity score for same prompt pairs:", vgg_l2_same_mean)
print("Standard deviation of the L2 Norm similarity scores for same prompt pairs:", vgg_l2_same_std)
print("Mean cosine similarity score for same prompt pairs:", vgg_cosine_same_mean)
print("Standard deviation of the cosine similarity scores for same prompt pairs:", vgg_cosine_same_std)

Mean L2 Norm similarity score for same prompt pairs: 0.6920358041922251
Standard deviation of the L2 Norm similarity scores for same prompt pairs: 0.0809709296937374
Mean cosine similarity score for same prompt pairs: 0.49793243904908496
Standard deviation of the cosine similarity scores for same prompt pairs: 0.1689909470863051


# Using CLIP Model's dense representations 

In [105]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import glob
import os

# Load the OpenAI CLIP Model
print('Loading CLIP Model...')
model = SentenceTransformer('clip-ViT-B-32')

Loading CLIP Model...


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/604 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/961k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

In [106]:
def get_clip_encoding(img1, img2):
  encoded_image = model.encode([img1, img2], batch_size=2, convert_to_tensor=True, show_progress_bar=True)
  return encoded_image

In [109]:
import copy
def clip_similarity(_images):
    l2_sim_scores = []
    cosine_sim_scores = []
    inception_scores = []

    for _i in range(0, len(_images)):
        images = _images[_i]
        for i in range(0, len(images)):
            for j in range(i + 1, len(images)):
                img1 = copy.deepcopy(images[i])
                img2 = copy.deepcopy(images[j])
                encoded_image = get_clip_encoding(img1, img2)
                l2_sim = similarity(encoded_image[0].cpu(), encoded_image[1].cpu())
                # Cosine similarity
                cosine_sim = get_cosine_sim(encoded_image[0].cpu(), encoded_image[1].cpu())
                l2_sim_scores.append(l2_sim)
                cosine_sim_scores.append(cosine_sim)

    return np.mean(np.array(l2_sim_scores)), np.std(np.array(l2_sim_scores)), np.mean(np.array(cosine_sim_scores)), np.std(np.array(cosine_sim_scores))

In [110]:
clip_same_mean, clip_same_std, clip_cosine_same_mean, clip_cosine_same_std = clip_similarity(same_images_list)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [111]:
print("Mean L2 Norm similarity score for same prompt pairs:", clip_same_mean)
print("Standard deviation of the L2 Norm similarity scores for same prompt pairs:", clip_same_std)
print("Mean cosine similarity score for same prompt pairs:", clip_cosine_same_mean)
print("Standard deviation of the cosine similarity scores for same prompt pairs:", clip_cosine_same_std)

Mean L2 Norm similarity score for same prompt pairs: 0.35136914838637623
Standard deviation of the L2 Norm similarity scores for same prompt pairs: 0.14960272981857115
Mean cosine similarity score for same prompt pairs: 0.7784485160594895
Standard deviation of the cosine similarity scores for same prompt pairs: 0.09833458326102153
